https://medium.com/analytics-vidhya/recognizing-over-50-candlestick-patterns-with-python-4f02a1822cb5

In [1]:
import numpy as np
import pandas as pd
import talib
from itertools import compress

In [2]:
stockFile = f'.\\data\\yf_stocks\\A.csv'

# Data Loading

In [3]:
historical_dict = pd.read_csv(stockFile)
historical_dict['VolumeFrom'] = historical_dict['Volume'].shift()

cols = [ 'Date', 'Close', 'High', 'Low', 'Open', 'VolumeFrom', 'Volume', 'Adj Close' ]
historical_dict = historical_dict[cols]

historical_dict.rename(columns = {'Date': 'date', 'Close': 'close', 'High': 'high', 
                                  'Low': 'low', 'Open': 'open', 'VolumeFrom': 'volumefrom',
                                  'Volume': 'volumeto', 'Adj Close': 'closefrom' }, inplace = True)

historical_dict

,date,close,high,low,open,volumefrom,volumeto,closefrom
0,2017-01-03,46.490002,46.750000,45.740002,45.930000,NaN,1739600,44.367439
1,2017-01-04,47.099998,47.380001,46.820000,46.930000,1739600.0,1821300,44.949593
2,2017-01-05,46.540001,47.070000,46.360001,47.049999,1821300.0,1503700,44.415161
3,2017-01-06,47.990002,48.070000,46.560001,46.630001,1503700.0,2883400,45.798954
4,2017-01-09,48.139999,48.560001,47.910000,48.009998,2883400.0,2575300,45.942108
...,...,...,...,...,...,...,...,...
1507,2022-12-28,148.089996,151.369995,147.639999,149.779999,879500.0,784300,147.869461
1508,2022-12-29,151.089996,151.899994,149.020004,149.610001,784300.0,854400,150.865005
1509,2022-12-30,149.649994,149.850006,147.830002,149.759995,854400.0,699800,149.649994
1510,2023-01-03,150.039993,153.130005,148.470001,151.960007,699800.0,1414300,150.039993


# Starting Analysis

In [4]:
candle_names = talib.get_function_groups()['Pattern Recognition']

In [5]:
candle_rankings = {
        "CDL3LINESTRIKE_Bull": 1,
        "CDL3LINESTRIKE_Bear": 2,
        "CDL3BLACKCROWS_Bull": 3,
        "CDL3BLACKCROWS_Bear": 3,
        "CDLEVENINGSTAR_Bull": 4,
        "CDLEVENINGSTAR_Bear": 4,
        "CDLTASUKIGAP_Bull": 5,
        "CDLTASUKIGAP_Bear": 5,
        "CDLINVERTEDHAMMER_Bull": 6,
        "CDLINVERTEDHAMMER_Bear": 6,
        "CDLMATCHINGLOW_Bull": 7,
        "CDLMATCHINGLOW_Bear": 7,
        "CDLABANDONEDBABY_Bull": 8,
        "CDLABANDONEDBABY_Bear": 8,
        "CDLBREAKAWAY_Bull": 10,
        "CDLBREAKAWAY_Bear": 10,
        "CDLMORNINGSTAR_Bull": 12,
        "CDLMORNINGSTAR_Bear": 12,
        "CDLPIERCING_Bull": 13,
        "CDLPIERCING_Bear": 13,
        "CDLSTICKSANDWICH_Bull": 14,
        "CDLSTICKSANDWICH_Bear": 14,
        "CDLTHRUSTING_Bull": 15,
        "CDLTHRUSTING_Bear": 15,
        "CDLINNECK_Bull": 17,
        "CDLINNECK_Bear": 17,
        "CDL3INSIDE_Bull": 20,
        "CDL3INSIDE_Bear": 56,
        "CDLHOMINGPIGEON_Bull": 21,
        "CDLHOMINGPIGEON_Bear": 21,
        "CDLDARKCLOUDCOVER_Bull": 22,
        "CDLDARKCLOUDCOVER_Bear": 22,
        "CDLIDENTICAL3CROWS_Bull": 24,
        "CDLIDENTICAL3CROWS_Bear": 24,
        "CDLMORNINGDOJISTAR_Bull": 25,
        "CDLMORNINGDOJISTAR_Bear": 25,
        "CDLXSIDEGAP3METHODS_Bull": 27,
        "CDLXSIDEGAP3METHODS_Bear": 26,
        "CDLTRISTAR_Bull": 28,
        "CDLTRISTAR_Bear": 76,
        "CDLGAPSIDESIDEWHITE_Bull": 46,
        "CDLGAPSIDESIDEWHITE_Bear": 29,
        "CDLEVENINGDOJISTAR_Bull": 30,
        "CDLEVENINGDOJISTAR_Bear": 30,
        "CDL3WHITESOLDIERS_Bull": 32,
        "CDL3WHITESOLDIERS_Bear": 32,
        "CDLONNECK_Bull": 33,
        "CDLONNECK_Bear": 33,
        "CDL3OUTSIDE_Bull": 34,
        "CDL3OUTSIDE_Bear": 39,
        "CDLRICKSHAWMAN_Bull": 35,
        "CDLRICKSHAWMAN_Bear": 35,
        "CDLSEPARATINGLINES_Bull": 36,
        "CDLSEPARATINGLINES_Bear": 40,
        "CDLLONGLEGGEDDOJI_Bull": 37,
        "CDLLONGLEGGEDDOJI_Bear": 37,
        "CDLHARAMI_Bull": 38,
        "CDLHARAMI_Bear": 72,
        "CDLLADDERBOTTOM_Bull": 41,
        "CDLLADDERBOTTOM_Bear": 41,
        "CDLCLOSINGMARUBOZU_Bull": 70,
        "CDLCLOSINGMARUBOZU_Bear": 43,
        "CDLTAKURI_Bull": 47,
        "CDLTAKURI_Bear": 47,
        "CDLDOJISTAR_Bull": 49,
        "CDLDOJISTAR_Bear": 51,
        "CDLHARAMICROSS_Bull": 50,
        "CDLHARAMICROSS_Bear": 80,
        "CDLADVANCEBLOCK_Bull": 54,
        "CDLADVANCEBLOCK_Bear": 54,
        "CDLSHOOTINGSTAR_Bull": 55,
        "CDLSHOOTINGSTAR_Bear": 55,
        "CDLMARUBOZU_Bull": 71,
        "CDLMARUBOZU_Bear": 57,
        "CDLUNIQUE3RIVER_Bull": 60,
        "CDLUNIQUE3RIVER_Bear": 60,
        "CDL2CROWS_Bull": 61,
        "CDL2CROWS_Bear": 61,
        "CDLBELTHOLD_Bull": 62,
        "CDLBELTHOLD_Bear": 63,
        "CDLHAMMER_Bull": 65,
        "CDLHAMMER_Bear": 65,
        "CDLHIGHWAVE_Bull": 67,
        "CDLHIGHWAVE_Bear": 67,
        "CDLSPINNINGTOP_Bull": 69,
        "CDLSPINNINGTOP_Bear": 73,
        "CDLUPSIDEGAP2CROWS_Bull": 74,
        "CDLUPSIDEGAP2CROWS_Bear": 74,
        "CDLGRAVESTONEDOJI_Bull": 77,
        "CDLGRAVESTONEDOJI_Bear": 77,
        "CDLHIKKAKEMOD_Bull": 82,
        "CDLHIKKAKEMOD_Bear": 81,
        "CDLHIKKAKE_Bull": 85,
        "CDLHIKKAKE_Bear": 83,
        "CDLENGULFING_Bull": 84,
        "CDLENGULFING_Bear": 91,
        "CDLMATHOLD_Bull": 86,
        "CDLMATHOLD_Bear": 86,
        "CDLHANGINGMAN_Bull": 87,
        "CDLHANGINGMAN_Bear": 87,
        "CDLRISEFALL3METHODS_Bull": 94,
        "CDLRISEFALL3METHODS_Bear": 89,
        "CDLKICKING_Bull": 96,
        "CDLKICKING_Bear": 102,
        "CDLDRAGONFLYDOJI_Bull": 98,
        "CDLDRAGONFLYDOJI_Bear": 98,
        "CDLCONCEALBABYSWALL_Bull": 101,
        "CDLCONCEALBABYSWALL_Bear": 101,
        "CDL3STARSINSOUTH_Bull": 103,
        "CDL3STARSINSOUTH_Bear": 103,
        "CDLDOJI_Bull": 104,
        "CDLDOJI_Bear": 104,
        'CDLLONGLINE_Bull': 105,
        'CDLLONGLINE_Bear': 106,
        'CDLSHORTLINE_Bull': 107,
        'CDLSHORTLINE_Bear': 108,
        'CDLSTALLEDPATTERN_Bull': 109,
        'CDLSTALLEDPATTERN_Bear': 110,
        'CDLCOUNTERATTACK_Bull': 111,
        'CDLCOUNTERATTACK_Bear': 112,
    }

In [6]:
df = historical_dict.drop(['closefrom'], axis=1)

In [7]:
# extract OHLC 
op = df['open']
hi = df['high']
lo = df['low']
cl = df['close']
# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [8]:
df

,date,close,high,low,open,volumefrom,volumeto,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,...,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
0,2017-01-03,46.490002,46.750000,45.740002,45.930000,NaN,1739600,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017-01-04,47.099998,47.380001,46.820000,46.930000,1739600.0,1821300,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017-01-05,46.540001,47.070000,46.360001,47.049999,1821300.0,1503700,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017-01-06,47.990002,48.070000,46.560001,46.630001,1503700.0,2883400,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017-01-09,48.139999,48.560001,47.910000,48.009998,2883400.0,2575300,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,2022-12-28,148.089996,151.369995,147.639999,149.779999,879500.0,784300,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1508,2022-12-29,151.089996,151.899994,149.020004,149.610001,784300.0,854400,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1509,2022-12-30,149.649994,149.850006,147.830002,149.759995,854400.0,699800,0,0,0,...,0,0,0,100,0,0,0,0,0,0
1510,2023-01-03,150.039993,153.130005,148.470001,151.960007,699800.0,1414300,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df['candlestick_pattern'] = np.nan
df['candlestick_match_count'] = np.nan
for index, row in df.iterrows():

    # no pattern found
    if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
        df.loc[index,'candlestick_pattern'] = "NO_PATTERN"
        df.loc[index, 'candlestick_match_count'] = 0
    # single pattern found
    elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
        # bull pattern 100 or 200
        if any(row[candle_names].values > 0):
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
            df.loc[index, 'candlestick_pattern'] = pattern
            df.loc[index, 'candlestick_match_count'] = 1
        # bear pattern -100 or -200
        else:
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
            df.loc[index, 'candlestick_pattern'] = pattern
            df.loc[index, 'candlestick_match_count'] = 1
    # multiple patterns matched -- select best performance
    else:
        # filter out pattern names from bool list of values
        patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
        container = []
        for pattern in patterns:
            if row[pattern] > 0:
                container.append(pattern + '_Bull')
            else:
                container.append(pattern + '_Bear')
        rank_list = [candle_rankings[p] for p in container]
        if len(rank_list) == len(container):
            rank_index_best = rank_list.index(min(rank_list))
            df.loc[index, 'candlestick_pattern'] = container[rank_index_best]
            df.loc[index, 'candlestick_match_count'] = len(container)
# clean up candle columns
df.drop(candle_names, axis = 1, inplace = True)

In [10]:
df

,date,close,high,low,open,volumefrom,volumeto,candlestick_pattern,candlestick_match_count
0,2017-01-03,46.490002,46.750000,45.740002,45.930000,NaN,1739600,NO_PATTERN,0.0
1,2017-01-04,47.099998,47.380001,46.820000,46.930000,1739600.0,1821300,NO_PATTERN,0.0
2,2017-01-05,46.540001,47.070000,46.360001,47.049999,1821300.0,1503700,NO_PATTERN,0.0
3,2017-01-06,47.990002,48.070000,46.560001,46.630001,1503700.0,2883400,NO_PATTERN,0.0
4,2017-01-09,48.139999,48.560001,47.910000,48.009998,2883400.0,2575300,NO_PATTERN,0.0
...,...,...,...,...,...,...,...,...,...
1507,2022-12-28,148.089996,151.369995,147.639999,149.779999,879500.0,784300,CDL3LINESTRIKE_Bull,2.0
1508,2022-12-29,151.089996,151.899994,149.020004,149.610001,784300.0,854400,CDLLONGLINE_Bull,1.0
1509,2022-12-30,149.649994,149.850006,147.830002,149.759995,854400.0,699800,CDLLONGLEGGEDDOJI_Bull,6.0
1510,2023-01-03,150.039993,153.130005,148.470001,151.960007,699800.0,1414300,NO_PATTERN,0.0


In [12]:
df.to_csv('./output.csv')

# Candle_stick using plotly

In [15]:
from plotly.offline import plot
import plotly.graph_objs as go
import pandas as pd

df = pd.read_csv('./output.csv')

o = df['open'].astype(float)
h = df['high'].astype(float)
l = df['low'].astype(float)
c = df['close'].astype(float)

trace = go.Candlestick(
            open=o,
            high=h,
            low=l,
            close=c)
data = [trace]

plot(data, filename='go_candle1.html')

'go_candle1.html'

# Pattern (pick_best_rank)

In [14]:
df['candlestick_pattern'] = np.nan
df['candlestick_match_count'] = np.nan
for index, row in df.iterrows():

    # no pattern found
    if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
        df.loc[index,'candlestick_pattern'] = "NO_PATTERN"
        df.loc[index, 'candlestick_match_count'] = 0
    # single pattern found
    elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
        # bull pattern 100 or 200
        if any(row[candle_names].values > 0):
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
            df.loc[index, 'candlestick_pattern'] = pattern
            df.loc[index, 'candlestick_match_count'] = 1
        # bear pattern -100 or -200
        else:
            pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
            df.loc[index, 'candlestick_pattern'] = pattern
            df.loc[index, 'candlestick_match_count'] = 1
    # multiple patterns matched -- select best performance
    else:
        # filter out pattern names from bool list of values
        patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
        container = []
        for pattern in patterns:
            if row[pattern] > 0:
                container.append(pattern + '_Bull')
            else:
                container.append(pattern + '_Bear')
        rank_list = [candle_rankings[p] for p in container]
        if len(rank_list) == len(container):
            rank_index_best = rank_list.index(min(rank_list))
            df.loc[index, 'candlestick_pattern'] = container[rank_index_best]
            df.loc[index, 'candlestick_match_count'] = len(container)
# clean up candle columns
df.drop(candle_names, axis = 1, inplace = True)

KeyError: "None of [Index(['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE',\n       'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS',\n       'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY',\n       'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK',\n       'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI',\n       'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR',\n       'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER',\n       'CDLHANGINGMAN', 'CDLHARAMI', 'CDLHARAMICROSS', 'CDLHIGHWAVE',\n       'CDLHIKKAKE', 'CDLHIKKAKEMOD', 'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS',\n       'CDLINNECK', 'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH',\n       'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU',\n       'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR',\n       'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS',\n       'CDLSEPARATINGLINES